In [188]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/victorsabino36/Documents/projetos_dev/credenciais/bigquery_cre.json"
import pandas as pd
from google.cloud import bigquery
import pandas as pd
dataset_id = "mythical-lyceum-450717-f6.transporte_log"
client = bigquery.Client()

In [189]:
try:
    
    # Define a consulta
    query = """
    SELECT * 
    FROM `mythical-lyceum-450717-f6.transporte_log.entregas`
    LIMIT 10
    """
    # Executa a consulta e carrega os resultados em um DataFrame do Pandas
    df = client.query(query).to_dataframe()
except Exception as e:
    print(f"Erro: {e}")



In [190]:
def resumo_entregas(client, data_inicio, data_fim):
    query = f""" 
SELECT 
    LOWER(status) AS status, 
    COUNT(*) AS total
FROM 
    `mythical-lyceum-450717-f6.transporte_log.entregas`
WHERE 
    DATE(data_entrega) BETWEEN DATE('{data_inicio}') AND DATE('{data_fim}')
GROUP BY 
    LOWER(status)
    """
    return client.query(query).to_dataframe()

def faturamento_total(client, data_inicio, data_fim):
    query = f"""
    SELECT SUM(valor) AS total_faturado 
    FROM `mythical-lyceum-450717-f6.transporte_log.entregas`
    WHERE 
        DATE(data_entrega) BETWEEN DATE('{data_inicio}') AND DATE('{data_fim}')
    """
    return client.query(query).to_dataframe()

def principais_motivos_cancelamentos(client, data_inicio, data_fim):
    query = f"""
    SELECT 
        LOWER(motivo_cancelamento) AS motivo_cancelamento, 
        COUNT(*) AS total_cancelamentos 
    FROM 
        `mythical-lyceum-450717-f6.transporte_log.entregas`
    WHERE 
        motivo_cancelamento IS NOT NULL 
        AND motivo_cancelamento != ''
        AND DATE(data_entrega) BETWEEN DATE('{data_inicio}') AND DATE('{data_fim}')
    GROUP BY 
        LOWER(motivo_cancelamento)
    ORDER BY 
        total_cancelamentos DESC;
    """
    return client.query(query).to_dataframe()

def get_tabela_entregas(client, data_inicio, data_fim):
    query = f"""
    SELECT 
        e.id_entrega, 
        m.id_motorista, 
        m.nome, 
        e.status
    FROM `mythical-lyceum-450717-f6.transporte_log.entregas` e
    JOIN `mythical-lyceum-450717-f6.transporte_log.motoristas` m 
    ON e.id_motorista = m.id_motorista
    WHERE 
        e.status IN ('entregue', 'cancelado')
        AND DATE(e.data_entrega) BETWEEN DATE('{data_inicio}') AND DATE('{data_fim}')
    ORDER BY e.status, m.nome;
    """
    return client.query(query).to_dataframe()


In [191]:
data_inicio = "2025-02-01"
data_fim = "2025-02-28"

df_resumo = resumo_entregas(client, data_inicio, data_fim)
df_faturamento = faturamento_total(client, data_inicio, data_fim)
df_motivos = principais_motivos_cancelamentos(client, data_inicio, data_fim)
df_tabela = get_tabela_entregas(client, data_inicio, data_fim)

In [192]:
df_resumo.head()

,status,total
0,pendente,28
1,cancelado,24
2,entregue,28


In [193]:
df_faturamento.head()


,total_faturado
0,216270.5


In [194]:
df_tabela.head()


,id_entrega,id_motorista,nome,status
0,77,30,Adam Davis,cancelado
1,11,34,Andres Hickman,cancelado
2,46,29,Arthur Decker,cancelado
3,23,13,Benjamin Ramirez,cancelado
4,22,21,Bill Huff,cancelado


In [195]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors

# Criar e salvar os gráficos como imagens
def gerar_graficos():
    # Gráfico 1: Total de entregas por status
    plt.figure(figsize=(6,4))
    sns.barplot(data=df_resumo, x='status', y='total', palette='viridis')
    plt.title('Total de Entregas por Status')
    plt.xlabel('Status')
    plt.ylabel('Total')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("total_entregas.png", bbox_inches='tight')  
    plt.close()
    """
    # Gráfico 2: Entregas realizadas vs canceladas (Pizza)
    df_pizza = df_resumo_entregas[df_resumo_entregas['status'].isin(['entregue', 'cancelado'])]
    plt.figure(figsize=(5,5))
    plt.pie(df_pizza['total'], labels=df_pizza['status'], autopct='%1.1f%%', colors=['blue', 'red'], startangle=90)
    plt.title('Proporção de Entregas Realizadas vs Canceladas')
    plt.savefig("pizza_entregas.png", bbox_inches='tight')
    plt.close()
    """

    # Gráfico 3: Motivos de Cancelamento
    plt.figure(figsize=(6,4))
    sns.barplot(data=df_motivos, x='motivo_cancelamento', y='total_cancelamentos', palette='viridis')
    plt.title('Principais Motivos de Cancelamento')
    plt.xlabel('Motivo de Cancelamento')
    plt.ylabel('Total de Cancelamentos')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("motivos_cancelamento.png", bbox_inches='tight')
    plt.close()


# Função para gerar o PDF 
def gerar_pdf(nome_arquivo="relatorios_entregas.pdf"):
    c = canvas.Canvas(nome_arquivo, pagesize=letter)
    width, height = letter  # Dimensão da página
    
    # Título
    c.setFont("Helvetica-Bold", 16)
    c.drawString(170, height - 80, "Relatório de entregas dE FEVEREIRO")

    # Faturamento Total
    c.setFont("Helvetica", 12)
    faturamento_texto = f"Faturamento Total: R$ {df_faturamento.iloc[0, 0]:,.2f}"
    c.drawString(200, height - 150, faturamento_texto)

    # Adicionar gráficos ao PDF
    imagens = ["total_entregas.png", "motivos_cancelamento.png"]
    y_position = height - 450  # Ajusta posição inicial dos gráficos

    for img in imagens:
        if y_position < 50:  # Se não houver espaço, cria nova página
            c.showPage()
            y_position = height - 100
        c.drawImage(ImageReader(img), 100, y_position, width=400, height=250, preserveAspectRatio=True, mask='auto')
        y_position -= 250  # Mover para a próxima posição

    # Criar uma nova página para a tabela
    c.showPage()
    c.setFont("Helvetica-Bold", 12)
    c.drawString(240, height - 17, "Tabela de Entregas:")

    # Criar tabela
    data = [["ID Motorista", "Nome", "Status"]]  # Cabeçalho
    for _, row in df_tabela[['id_motorista', 'nome', 'status']].head(10).iterrows():
        data.append([row['id_motorista'], row['nome'], row['status']])

    table = Table(data, colWidths=[100, 100, 100])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 10),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))

    table.wrapOn(c, width, height)
    table.drawOn(c, 150, height - 250)  # Ajusta posição da tabela

    c.save()
    print(f"PDF gerado: {nome_arquivo}") 


# Gerar os gráficos
gerar_graficos()

# Gerar o PDF final
gerar_pdf()


/var/folders/mt/2h29lwb91pv7n5c03zf4q5hc0000gp/T/ipykernel_23367/2900834304.py:14: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=df_resumo, x='status', y='total', palette='viridis')


/var/folders/mt/2h29lwb91pv7n5c03zf4q5hc0000gp/T/ipykernel_23367/2900834304.py:34: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=df_motivos, x='motivo_cancelamento', y='total_cancelamentos', palette='viridis')


PDF gerado: relatorios_entregas.pdf


In [196]:
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# Configurações do e-mail
EMAIL_REMETENTE = "victorsabino36@gmail.com"  # Substituir pelo seu e-mail
EMAIL_SENHA = "umpl whee ckqp clnh"  # Substituir pela senha do app (Gmail) ou senha normal (Outlook)
EMAIL_DESTINATARIO = "victorsabino36@outlook.com"  # Substituir pelo e-mail do destinatário

# Criar e-mail
msg = MIMEMultipart()
msg["From"] = EMAIL_REMETENTE
msg["To"] = EMAIL_DESTINATARIO
msg["Subject"] = "Relatório de Entregas"

# Corpo do e-mail
corpo_email = "Segue em anexo o relatório das entregas."
msg.attach(MIMEText(corpo_email, "plain"))

# Anexar o PDF
nome_arquivo = "relatorios_entregas.pdf"
caminho_arquivo = os.path.join(os.getcwd(), nome_arquivo)

with open(caminho_arquivo, "rb") as anexo:
    parte = MIMEBase("application", "octet-stream")
    parte.set_payload(anexo.read())
    encoders.encode_base64(parte)
    parte.add_header("Content-Disposition", f"attachment; filename={nome_arquivo}")
    msg.attach(parte)

# Configurar servidor SMTP (Gmail)
try:
    servidor = smtplib.SMTP("smtp.gmail.com", 587)
    servidor.starttls()
    servidor.login(EMAIL_REMETENTE, EMAIL_SENHA)
    texto_email = msg.as_string()
    servidor.sendmail(EMAIL_REMETENTE, EMAIL_DESTINATARIO, texto_email)
    servidor.quit()
    print("✅ E-mail enviado com sucesso!")
except Exception as e:
    print(f"❌ Erro ao enviar e-mail: {e}")


✅ E-mail enviado com sucesso!
